In [5]:
# =============================================================================
# STEP 4: VISUAL AI DEMO – YOLOv8 PARKING MANAGEMENT VISUALIZATION
# =============================================================================

# !pip install ultralytics 
# opencv-python numpy

print("\n🎥 STEP 4: VISUAL AI DEMO – YOLOv8 PARKING MANAGEMENT VISUALIZATION")

import cv2
import json
import numpy as np
from ultralytics import YOLO
from datetime import datetime
import os

class ParkingVisualizer:
    def __init__(self, model_path, region_json, video_path, output_path):
        """
        YOLOv8-based Parking Visualization System
        """
        self.model = YOLO(model_path)
        self.video_path = video_path
        self.output_path = output_path
        
        with open(region_json, 'r') as f:
            self.regions = json.load(f)

        self.colors = {
            'occupied': (0, 0, 255),
            'available': (0, 255, 0)
        }

    def is_inside_region(self, point, polygon):
        """Check if a detected object's center point lies inside a polygon region."""
        return cv2.pointPolygonTest(np.array(polygon, np.int32), point, False) >= 0

    def run(self, show_live=False):
        """Run visualization process."""
        cap = cv2.VideoCapture(self.video_path)
        if not cap.isOpened():
            print("❌ Error: Unable to open video file.")
            return
        
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        os.makedirs(os.path.dirname(self.output_path), exist_ok=True)
        out = cv2.VideoWriter(
            self.output_path,
            cv2.VideoWriter_fourcc(*'mp4v'),
            fps,
            (frame_width, frame_height)
        )

        print("🚗 Processing video frames for parking visualization...")
        frame_count = 0
        total_spots = len(self.regions)
        start_time = datetime.now()

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            detections = self.model(frame, verbose=False)
            boxes = detections[0].boxes.xyxy.cpu().numpy() if detections and detections[0].boxes else []

            region_status = {str(i): 'available' for i in range(len(self.regions))}
            for box in boxes:
                x1, y1, x2, y2 = map(int, box[:4])
                cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)
                for i, region in enumerate(self.regions):
                    if self.is_inside_region((cx, cy), region['points']):
                        region_status[str(i)] = 'occupied'

            occupied = list(region_status.values()).count('occupied')
            available = total_spots - occupied
            occupancy_rate = (occupied / total_spots) * 100

            for i, region in enumerate(self.regions):
                pts = np.array(region['points'], np.int32)
                status = region_status[str(i)]
                color = self.colors[status]
                cv2.polylines(frame, [pts], True, color, 2)
                cv2.putText(frame, status.upper(),
                            (pts[0][0], pts[0][1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            cv2.rectangle(frame, (10, 10), (350, 70), (50, 50, 50), -1)
            cv2.putText(frame, f"Total Spots: {total_spots}", (20, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"Available: {available}  Occupied: {occupied}", (20, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            cv2.putText(frame, f"Occupancy: {occupancy_rate:.1f}%", (200, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            out.write(frame)
            frame_count += 1

            # Show live video only if environment supports GUI
            if show_live:
                try:
                    cv2.imshow("Smart Parking Visualization", frame)
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                except cv2.error:
                    print("⚠️ GUI not supported — skipping live display.")
                    show_live = False  # disable further attempts

        if frame_count % 50 == 0:
            print(f"🧩 Processed {frame_count} frames...")

        cap.release()
        out.release()

        # Safely close any OpenCV windows (if GUI available)
        if show_live:
            try:
                cv2.destroyAllWindows()
            except cv2.error:
                print("⚠️ GUI not supported — skipping window cleanup.")

        duration = (datetime.now() - start_time).seconds
        print(f"✅ Visualization complete — saved to {self.output_path}")
        print(f"🖼️ Processed {frame_count} frames in {duration}s")
        print(f"📊 Final Occupancy Rate: {occupancy_rate:.1f}% ({occupied}/{total_spots} occupied)")

# =============================================================================
# Example Usage
# =============================================================================

model_path = "models/yolov8n.pt"
region_json = "data/parking_zones.json"
video_path = "data/sample_video.mp4"
output_path = "output/parking_visual.mp4"

visualizer = ParkingVisualizer(model_path, region_json, video_path, output_path)
visualizer.run(show_live=False)  # ✅ Set to True only if your environment supports OpenCV windows



🎥 STEP 4: VISUAL AI DEMO – YOLOv8 PARKING MANAGEMENT VISUALIZATION
🚗 Processing video frames for parking visualization...
✅ Visualization complete — saved to output/parking_visual.mp4
🖼️ Processed 206 frames in 32s
📊 Final Occupancy Rate: 0.0% (0/2 occupied)
